In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, Dense, LSTM, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import random
import warnings

warnings.filterwarnings(action='ignore')

In [3]:
# 데이터 불러오기
df1 = pd.read_csv('../data/review-rating1.csv')
df2 = pd.read_csv('../data/review-rating2.csv')
df3 = pd.read_csv('../data/review-rating3.csv')
df4 = pd.read_csv('../data/review-rating4.csv')
df = pd.concat([df1, df2, df3, df4])

# 중복값 제거
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

# 랜덤으로 200개의 리뷰 가져오기
random_indices = random.sample(range(len(df)), k=500)
temp = df.loc[random_indices]
temp.drop_duplicates(subset=['영화ID'], inplace=True)
temp.reset_index(inplace=True, drop=True)
random_indices = random.sample(range(len(temp)), k=200)
df_random = temp.loc[random_indices]
df_random.reset_index(inplace=True, drop=True)
df_random

,실제평점,리뷰,영화ID,영화명,평균평점
0,2.0,아기한테 수면제 탄 분유 쳐먹이는게 용서가안된다. 끔찍하다. 불쌍한 아가,2788,해피엔드,8.39
1,1.0,돈과시간 아깝다는 생각에 치를 떨기 딱 좋은 영화.,3748,나의 사랑 나의 신부,8.39
2,10.0,진짜 역대급입니다...,4968,수부라 게이트,6.91
3,10.0,초딩때 비디오로 5번은 본거같다 ㅎㅎ,7474,미세스 다웃파이어,9.39
4,8.0,12세 관람가에서 약간 의아했그..무서운거 잘보시는 분들에게는 비추 처음부터 끝까지...,7384,위자,7.73
...,...,...,...,...,...
195,7.0,"내가추리력이 좋은건지는모르겠지만, 첨부터 스토리가너무뻔했다",1769,더블 크라임,7.95
196,1.0,에고... 피해본 가수는 어떻하나요 ㅠ 안타깝네요 에고에고에고,666,생각의 여름,7.50
197,1.0,아우 OO아 말론 브란도의 초상화 가리지마라!!! 아놔!!!!!! (이 영화 보고 ...,9928,대부 : 끝나지 않은 이야기,6.85
198,10.0,눈 굴리기가 이경규 능가..그걸로 연기를 끝내는 신공. 넓은 미간도 사랑스러워...,775,평평남녀,5.80


In [4]:
movie = pd.read_csv('../data/movie_info.csv')
movie = movie[['영화ID', '배우']]
movie

,영화ID,배우
0,0,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형..."
1,1,"조 샐다나,샘 워싱턴,시고니 위버,우나 채플린,지오바니 리비시,스티븐 랭,케이트 윈..."
2,2,"톰 크루즈,마일즈 텔러,제니퍼 코넬리,존 햄,에드 해리스,글렌 포웰,제이 엘리스,그..."
3,3,"톰 홀랜드,젠데이아 콜먼,베네딕트 컴버배치,존 파브로,제이콥 배덜런,마리사 토메이,..."
4,4,"박해일,변요한,안성기,손현주,김성규,김성균,김향기,택연,공명,박지환,조재윤,윤제문,..."
...,...,...
8925,11278,"스타니슬라브 장,바호디르 무사에프,일벡 파이지에프,드미트리 엄,박 루슬란"
8926,11279,"김선영,은하,정두원,이미선,신안진,이무녕"
8927,11280,"제임스 프랭코,프리다 핀토,앤디 서키스,브라이언 콕스,톰 펠튼,존 리스고,데이빗 휴..."
8928,11281,"코토부키 미나코,미야노 마모루,코니시 카츠유키,카미야 히로시,사카모토 마아야,노지마..."


In [5]:
actors = pd.merge(df_random, movie, on='영화ID')
actors.rename({'배우':'actor', '평균평점':'rating'}, axis=1, inplace=True)
actors

,실제평점,리뷰,영화ID,영화명,rating,actor
0,2.0,아기한테 수면제 탄 분유 쳐먹이는게 용서가안된다. 끔찍하다. 불쌍한 아가,2788,해피엔드,8.39,"최민식,전도연,주진모,주현"
1,1.0,돈과시간 아깝다는 생각에 치를 떨기 딱 좋은 영화.,3748,나의 사랑 나의 신부,8.39,"조정석,신민아,윤정희,배성우,라미란,전무송,이시언,고규필,서강준,유하준,장다경,황정..."
2,10.0,진짜 역대급입니다...,4968,수부라 게이트,6.91,"피에르프란체스코 파...,엘리오 게르마노,클라우디오 아멘돌라,줄리아 고리에티,알레산..."
3,10.0,초딩때 비디오로 5번은 본거같다 ㅎㅎ,7474,미세스 다웃파이어,9.39,"로빈 윌리엄스,샐리 필드,피어스 브로스넌,하비 피어스타인,폴리 홀리데이"
4,8.0,12세 관람가에서 약간 의아했그..무서운거 잘보시는 분들에게는 비추 처음부터 끝까지...,7384,위자,7.73,"엘리자베스 리저,애너리즈 바쏘,룰루 윌슨,헨리 토마스,파커 맥,할리 찰튼,알렉시스 ..."
...,...,...,...,...,...,...
195,7.0,"내가추리력이 좋은건지는모르겠지만, 첨부터 스토리가너무뻔했다",1769,더블 크라임,7.95,"토미 리 존스,애슐리 쥬드,브루스 그린우드,안나베스 기쉬,로마 마피아"
196,1.0,에고... 피해본 가수는 어떻하나요 ㅠ 안타깝네요 에고에고에고,666,생각의 여름,7.50,"김예은,곽민규,오규철,한해인,신기환,서유성"
197,1.0,아우 OO아 말론 브란도의 초상화 가리지마라!!! 아놔!!!!!! (이 영화 보고 ...,9928,대부 : 끝나지 않은 이야기,6.85,"제랄드 랑뱅,체키 카료,다니엘 듀발,패트릭 카탈리포,프랑소와 레반탈,프란시스 레노드..."
198,10.0,눈 굴리기가 이경규 능가..그걸로 연기를 끝내는 신공. 넓은 미간도 사랑스러워...,775,평평남녀,5.80,"이태경,이한주,이봄,서갑숙,유민곤"


In [6]:
# 배우 리스트
temp_lis = []
for i, lis in enumerate(actors['actor']):
    X_lis = []
    X_lis = lis.split(',')
    temp_lis.append(X_lis)
actors['actor'] = temp_lis
actors

,실제평점,리뷰,영화ID,영화명,rating,actor
0,2.0,아기한테 수면제 탄 분유 쳐먹이는게 용서가안된다. 끔찍하다. 불쌍한 아가,2788,해피엔드,8.39,"[최민식, 전도연, 주진모, 주현]"
1,1.0,돈과시간 아깝다는 생각에 치를 떨기 딱 좋은 영화.,3748,나의 사랑 나의 신부,8.39,"[조정석, 신민아, 윤정희, 배성우, 라미란, 전무송, 이시언, 고규필, 서강준, ..."
2,10.0,진짜 역대급입니다...,4968,수부라 게이트,6.91,"[피에르프란체스코 파..., 엘리오 게르마노, 클라우디오 아멘돌라, 줄리아 고리에티..."
3,10.0,초딩때 비디오로 5번은 본거같다 ㅎㅎ,7474,미세스 다웃파이어,9.39,"[로빈 윌리엄스, 샐리 필드, 피어스 브로스넌, 하비 피어스타인, 폴리 홀리데이]"
4,8.0,12세 관람가에서 약간 의아했그..무서운거 잘보시는 분들에게는 비추 처음부터 끝까지...,7384,위자,7.73,"[엘리자베스 리저, 애너리즈 바쏘, 룰루 윌슨, 헨리 토마스, 파커 맥, 할리 찰튼..."
...,...,...,...,...,...,...
195,7.0,"내가추리력이 좋은건지는모르겠지만, 첨부터 스토리가너무뻔했다",1769,더블 크라임,7.95,"[토미 리 존스, 애슐리 쥬드, 브루스 그린우드, 안나베스 기쉬, 로마 마피아]"
196,1.0,에고... 피해본 가수는 어떻하나요 ㅠ 안타깝네요 에고에고에고,666,생각의 여름,7.50,"[김예은, 곽민규, 오규철, 한해인, 신기환, 서유성]"
197,1.0,아우 OO아 말론 브란도의 초상화 가리지마라!!! 아놔!!!!!! (이 영화 보고 ...,9928,대부 : 끝나지 않은 이야기,6.85,"[제랄드 랑뱅, 체키 카료, 다니엘 듀발, 패트릭 카탈리포, 프랑소와 레반탈, 프란..."
198,10.0,눈 굴리기가 이경규 능가..그걸로 연기를 끝내는 신공. 넓은 미간도 사랑스러워...,775,평평남녀,5.80,"[이태경, 이한주, 이봄, 서갑숙, 유민곤]"


In [7]:
X = actors['actor']
y = actors['rating']

In [10]:
# 정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

# 단어수
vocab_size = len(tokenizer.word_index)
vocab_size

# 텍스트를 숫자 시퀀스로 변환
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X) 
X = tokenizer.texts_to_sequences(X)

# 종속변수를 array로 변환
y = np.array(y)

In [12]:
for x in X:
    x = np.array(x)

In [13]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

# 모델 생성하는 함수
def create_model(embedding_dim=250, units=128):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    model.add(LSTM(units=units))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 모델 생성
model = KerasRegressor(build_fn=create_model, verbose=0)

# 하이퍼파라미터 탐색 영역을 정의
param_grid={
    'epochs': [10],
    'batch_size': [32, 64, 128],
    'embedding_dim': [150, 200, 250],
    'units': [64, 128, 256],
}

# 파라미터 검색
grid=GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

# 모델 학습
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=3)
grid_result = grid.fit(X, y, validation_split=0.2, callbacks=es)

# 결과 출력
print(grid_result.best_params_)
print(grid_result.best_score_)

ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, 

In [ ]:
# 모델 생성
model = Sequential()
model.add(Embedding(vocab_size, 250))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 모델 학습
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=3)
model.fit(X, y, batch_size=64, epochs=10, validation_split=0.2, callbacks=es)

In [ ]:
# 결과 시리즈로 저장
series = model.predict(X)
series